In [36]:
import math
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

## Load Data and Observation

In [37]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,id,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
0,0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,52,0,7515.0,1
1,1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,20,1,1756.0,0
2,2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,28,1,1311.0,0
3,3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,55,1,1443.0,0
4,4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,28,1,1542.0,0


In [38]:
# Fill NaN
labels = train['label']
# print(labels)
del train['label']

for key in train.keys():
    train[key].fillna(train[key].mean(),inplace=True)
    print(key)
for key in test.keys():
    test[key].fillna(test[key].mean(),inplace=True)

id
MO HLADR+ MFI (cells/ul)
Neu CD64+MFI (cells/ul)
CD3+T (cells/ul)
CD8+T (cells/ul)
CD4+T (cells/ul)
NK (cells/ul)
CD19+ (cells/ul)
CD45+ (cells/ul)
Age
Sex 0M1F
Mono CD64+MFI (cells/ul)


## Models

In [39]:
# K-folds is used to select paras
folds = 10
num = math.ceil(len(train)/folds)

### Random Forrest

In [40]:
estimators = [25, 50, 75, 100]
train_np = train.to_numpy()
test_np = test.to_numpy()
labels_np = labels.to_numpy().reshape(-1,1)

In [47]:
for e in estimators:
    start = 0
    acc = 0
    for i in range(folds):
        val_x = train[start:start+num]
        val_y = labels_np[start:start+num].ravel()
        train_x = np.vstack((train[:start], train[start+num:]))
        train_y = np.vstack((labels_np[:start], labels_np[start+num:])).ravel()
        clf = RandomForestClassifier(n_estimators=e)
        clf.fit(train_x,train_y)
        acc += clf.score(val_x,val_y)
        start = start+num
    acc /= folds
    print("Estimators:",e, "Acc:",acc)

Estimators: 25 Acc: 0.8722222222222223
Estimators: 50 Acc: 0.9111111111111112
Estimators: 75 Acc: 0.861111111111111
Estimators: 100 Acc: 0.8944444444444445


In [48]:
clf = RandomForestClassifier(n_estimators=75)
clf.fit(train,labels_np.ravel())
result = clf.predict(test)

In [49]:
out = pd.DataFrame()
out['id'] = test['id']
out['label'] = result
out.to_csv('submission.csv',index=None)